In [1]:
#Librearìa que busca el servicio de spak
import findspark

#Librerìas para gestionar el servicio
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

#Iniciación de sesion
findspark.init()

#Creamos la conf de spak y seteamos un nombre para la aplicación
sparkConf = SparkConf().setAppName("Convertir Parquet")
sc = SparkContext(conf=sparkConf)  #Creamos el contexto: Es como un string de conexion a unos contestos "x" y unas aplicaciones que estamos usando
spark = SparkSession(sc)

In [3]:
!hdfs dfs -ls

In [6]:
!hdfs dfs -copyFromLocal credit.csv /tmp/

copyFromLocal: `/tmp/credit.csv': File exists


In [9]:
#Creamos el dataframe con el archivo .csv
creditos = spark.read.load('/tmp/credit.csv',
                           format="csv",
                           sep=',',
                           inferSchema='true',
                           header='true')

En caso de no correr el siguiete comando, debe tenerse en cuenta que el parquet ya se encuentre instalado. En caso de no estar instalado debe ejecutarse el comando 

!pip install fastparquet

In [13]:
#La sigueinte instruccion particiona el archivo en un elemento, escribe en modo de sobreescritura (en caso que ya exista) 
#y lo crea con un nuevo formato, en este caso formato parquet
creditos.repartition(1).write.mode('overwrite').parquet('/tmp/credits')

In [14]:
!hdfs dfs -ls /tmp

Found 2 items
-rw-r--r--   1 root supergroup     133638 2021-05-17 19:29 /tmp/credit.csv
drwxr-xr-x   - root supergroup          0 2021-05-17 20:46 /tmp/credits


In [15]:
#Leer o cargar los datos en formato parquet
dfparquet = spark.read.parquet('/tmp/credits')

In [18]:
#Imprimir las 10 primeras filas usando parquet
dfparquet.select(['credit_history', 'purpose','amount']).show(10)

+--------------+----------+------+
|credit_history|   purpose|amount|
+--------------+----------+------+
|      critical|  radio/tv|  1169|
|        repaid|  radio/tv|  5951|
|      critical| education|  2096|
|        repaid| furniture|  7882|
|       delayed| car (new)|  4870|
|        repaid| education|  9055|
|        repaid| furniture|  2835|
|        repaid|car (used)|  6948|
|        repaid|  radio/tv|  3059|
|      critical| car (new)|  5234|
+--------------+----------+------+
only showing top 10 rows

